In [80]:
# Adding this snippet so the code can run on osx
from sys import platform as platform_name
if platform_name == "darwin":  
    import sys
    sys.path.append('//anaconda/lib/python3.5/site-packages/')
    # matplotlib is breaking 
    import matplotlib
    matplotlib.use('Agg')

from random import choice
from string import ascii_uppercase
import math
import time
from swalign import swalign
from scipy.stats import beta
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import simps
from numpy import trapz
import random
import operator
import itertools
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from sklearn.cluster import KMeans


# ****** functions ********

#read n .fna database files in the specified path
#set n = 0 to read all files
def ReadDataBase(_path, n):
    seqList = []
    from os import path
    files = os.listdir(_path) #makes a list of all files in folder
    i = 0
    j = 0
    for f in files:
        for seq_record in SeqIO.parse(_path + f, "fasta"): 
            seqList.append(seq_record.seq) # reads each file into a list
            j += 1
            if(n > 0):
                if(j > n-1):
                    i = n
                    break
        i += 1
        j = 0
        
        if(n > 0):
            if(i > n-1):
                break
                
    return seqList               

#creates dictionary with all permutations of length n 
#with repetition to index Feature Vector
def CreateDictionary(n):
    chars = "ACGT"
    arr = list(itertools.product(chars, repeat=n))
    
    D = {}
    i = 0

    for a in arr:
        D[''.join(a)] = i
        i += 1
        
    return D

#builds the feature vector for sequence using specified indexing dictionary
def FeatureVector(dictionary, sequence, n):    
    sLen = len(sequence)
    arr = [0]*4**n
    i = 0
    
    while(1):
        w = sequence[i:i+n]
        try:
            arr[D[w]] += 1
        except:
            i = i
        i += 1
        if(i+n > sLen):
            break
    
    return arr

#Reads the DB files and puts the information of the file in a array of strings
def readfile(filename):
    temp = open(filename, 'r').read().split('\n')
    return temp
    
    
#returns a random string of specified length
#length: strign length
def randomword(length):
    return (''.join(choice('ACGT') for i in range(0, length)))

#retuns an array of random strings
#size: how many strings there will be in the array
#lakeMinLen: min sequence length
#lakeMaxLen: max sequence length
def lakeString(size, lakeMinLen, lakeMaxLen):     
    lake_water = []
    for i in range(0, size):
        random.seed()
        #generates a random sequence length
        y = random.randint(lakeMinLen, lakeMaxLen)
        
        _str = randomword(y)
        lake_water.append(_str)
    return lake_water

def readTxtFile(filename):
    data = []
    fid = open(filename,'r')
    for line in fid:
        data.append(line)
    return data
        

# ******************************************************* main ****************************************************

use_pca = True
pca_components = 2

print("reading viruses...")
known_viruses = ReadDataBase("../database/virus/", 200)
print("reading bacterias...")
known_bacterias = ReadDataBase("../database/bact/", 200)
print("reading lake samples...")

# lake = ReadDataBase("../database/lake/", 50)
lake = readTxtFile("../database/lake.txt") 

print("finished reading data")

#matrix with all feature vectors
n = 4
D = CreateDictionary(n)
lake_matrix = []
virus_matrix = []
bact_matrix = []

for w in lake:
    arr = FeatureVector(D, str(w), n)
    arr = np.divide(np.array(arr), len(w))
    lake_matrix.append(arr)

for w in known_viruses:
    arr = FeatureVector(D, str(w), n)
    arr = np.divide(np.array(arr), len(w))
    virus_matrix.append(arr)

for w in known_bacterias:
    arr = FeatureVector(D, str(w), n)
    arr = np.divide(np.array(arr), len(w))
    bact_matrix.append(arr)

len_lake = len(lake)
len_viruses = len(known_viruses)
    
print("lake shape: ", np.matrix(lake_matrix).shape)
print("virus shape: ", np.matrix(virus_matrix).shape)
print("bacteria shape: ", np.matrix(bact_matrix).shape)

matrix = np.vstack((virus_matrix,bact_matrix))


### PCA
if use_pca:
    X = np.array(matrix)
    # PCA input: samples x features
    pca = PCA(n_components=pca_components)
    Xhat = pca.fit_transform(X)
    print("Percentage of represented variance: ", sum(pca.explained_variance_ratio_))

    data = np.transpose(Xhat)
    data_virus = data[:, :len_viruses]
    data_bact = data[:,len_viruses:]

#     ####### Plot results
#     if pca_components == 2:
#         plt.plot(data_virus[0], data_virus[1], 'go')
#         plt.plot(data_bact[0], data_bact[1], 'ro')
#     elif pca_components == 3:
#         fig = plt.figure()
#         ax = fig.add_subplot(111, projection='3d')
#         ax.scatter(data_virus[0], data_virus[1], data_virus[2], c='g', depthshade=False)
#         ax.scatter(data_bact[0], data_bact[1], data_bact[2], c='r', depthshade=False)
#     plt.show()



#### K MEANS CLUSTERING
if use_pca:
    data = np.array(Xhat)
else:
    data = np.array(matrix)
    
# training
estimator = KMeans(n_clusters=12)
estimator.fit(data)
training_labels = estimator.labels_

# testing
if use_pca:
    lake_matrix = pca.transform(lake_matrix)
estimated_labels = estimator.predict(lake_matrix)

right = 0
cnt_lbl = 0
for lbl in estimated_labels:
    labels_idx = np.where(training_labels == lbl)[0]
    cnt_virus = 0
    cnt_bact = 0
    for lblidx in labels_idx:
        if lblidx < len_viruses:
            cnt_virus += 1
        else:
            cnt_bact += 1
    if cnt_virus > cnt_bact and cnt_lbl < 100: # 100 for that sample
        right += 1
    if cnt_bact > cnt_virus and cnt_lbl >= 100:
        right += 1
    cnt_lbl += 1

print ("\nAccuraccy: ", right/len_lake)




# data_trans = np.transpose(data)
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# for i in range(len(data_in)):
#     if i < len_viruses:
#         if labels[i] == 0:
#             ax.scatter(data_trans[0,i], data_trans[1,i], data_trans[2,i], c='y', marker='o', depthshade=False)
#         elif labels[i] == 1:
#             ax.scatter(data_trans[0,i], data_trans[1,i], data_trans[2,i], c='y', marker='^', depthshade=False)
#     elif i >= len_viruses:
#         if labels[i] == 0:
#             ax.scatter(data_trans[0,i], data_trans[1,i], data_trans[2,i], c='b', marker='o', depthshade=False)
#         elif labels[i] == 1:
#             ax.scatter(data_trans[0,i], data_trans[1,i], data_trans[2,i], c='b', marker='^', depthshade=False)


plt.show()
# plt.savefig('myfig')

reading viruses...
reading bacterias...
reading lake samples...
finished reading data
lake shape:  (200, 256)
virus shape:  (200, 256)
bacteria shape:  (200, 256)
Percentage of represented variance:  0.399059612641

Accuraccy:  0.77
